In [1]:
from datetime import datetime
from pathlib import Path

import torch
import numpy as np
import pandas as pd

In [8]:
path = Path("/home/tri110414/nfs_home/yolo-pruebas/results/V9/fusion_methods")
# Number of files in the directory
files = list(path.glob("*"))
print(f"Number of files: {len(files)}")

Number of files: 48


Inspect csv of train results

In [8]:
csvs_to_analyze = {
    "yolov8_best": Path(r"runs_OWOD/20240313_1407_owod_t1_yolov8l_from_scratch/results.csv"),
    "yolov8_new_2132": Path(r"runs_OWOD/20250601_2132_owod_t1_yolov8l_from_scratch/results.csv"),
    "yolov8_new_2354": Path(r"runs_OWOD/20250603_2354_owod_t1_yolov8l_from_scratch/results.csv"),
    "yolov9": Path(r"runs_OWOD/20250526_2123_owod_t1_yolov9m_from_scratch/results.csv"),
    "yolov10": Path(r"runs_OWOD/20250525_1733_owod_t1_yolov10l_from_scratch/results.csv"),
    "yolov11l": Path(r"runs_OWOD/20250517_1728_owod_t1_yolo11l_from_scratch/results.csv"),
    "yolov11m": Path(r"runs_OWOD/20250601_2346_owod_t1_yolo11m_from_scratch/results.csv"),
    "yolov12": Path(r"runs_OWOD/20250523_0011_owod_t1_yolo12l_from_scratch/results.csv"),
}

In [9]:
# Load the csvs with pandas
all_results_df_dict = {}
for name, csv_path in csvs_to_analyze.items():
    if not csv_path.exists():
        print(f"CSV file {csv_path} does not exist. Skipping.")
        continue
    #df = pd.read_csv(csv_path, on_bad_lines='skip', encoding='utf-8', low_memory=False,)
    #df = pd.read_csv(csv_path, encoding='ISO-8859-1')
    #df = pd.read_csv(csv_path, encoding='utf-8', on_bad_lines='skip')
    df = pd.read_csv(csv_path, encoding='windows-1252')
    all_results_df_dict[name] = df

In [10]:
all_results_df_dict["yolov9"].keys()

Index(['epoch', 'time', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss',
       'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)',
       'metrics/mAP50-95(B)', 'val/box_loss', 'val/cls_loss', 'val/dfl_loss',
       'lr/pg0', 'lr/pg1', 'lr/pg2'],
      dtype='object')

In [11]:
def fitness(p, r, map_05, map_0595):
    """Model fitness as a weighted combination of metrics."""
    w = [0.0, 0.0, 0.1, 0.9]  # weights for [P, R, mAP@0.5, mAP@0.5:0.95]
    return np.dot(w, [p, r, map_05, map_0595])

# Example
fitness(p=0.8, r=0.7, map_05=0.75, map_0595=0.65)

0.66

In [12]:
# Print the epoch of the max values of the  'metrics/precision(B)', 'metrics/recall(B)', metrics/mAP50(B) and	metrics/mAP50-95(B)
for name, df in all_results_df_dict.items():
    try:
        print(f"{name}:")
        
        max_precision_epoch = df['metrics/precision(B)'].idxmax()
        max_recall_epoch = df['metrics/recall(B)'].idxmax()
        max_mAP50_epoch = df['metrics/mAP50(B)'].idxmax()
        max_mAP50_95_epoch = df['metrics/mAP50-95(B)'].idxmax()
        # Compute the fitness value for all rows and select the max
        df['fitness'] = df.apply(lambda row: fitness(row['metrics/precision(B)'], 
                                                      row['metrics/recall(B)'], 
                                                      row['metrics/mAP50(B)'], 
                                                      row['metrics/mAP50-95(B)']), axis=1)
        max_fitness_epoch = df['fitness'].idxmax()
        print(f" {max_precision_epoch} Max precision(B) with value {df['metrics/precision(B)'][max_precision_epoch]}")
        print(f" {max_recall_epoch} Max recall(B) with value {df['metrics/recall(B)'][max_recall_epoch]}")
        print(f" {max_mAP50_epoch} Max mAP50(B) with value {df['metrics/mAP50(B)'][max_mAP50_epoch]}")
        print(f" {max_mAP50_95_epoch} Max mAP50-95(B) with value {df['metrics/mAP50-95(B)'][max_mAP50_95_epoch]}")
        print(f" {max_fitness_epoch} Max fitness with value {df['fitness'][max_fitness_epoch]}")        
    except KeyError:
        # Try with these: '   metrics/precision(B)', '      metrics/recall(B)', '       metrics/mAP50(B)', '    metrics/mAP50-95(B)'
        max_precision_epoch = df['   metrics/precision(B)'].idxmax()
        max_recall_epoch = df['      metrics/recall(B)'].idxmax()
        max_mAP50_epoch = df['       metrics/mAP50(B)'].idxmax()
        max_mAP50_95_epoch = df['    metrics/mAP50-95(B)'].idxmax()
        # Compute the fitness value for all rows and select the max
        df['fitness'] = df.apply(lambda row: fitness(row['   metrics/precision(B)'], 
                                                      row['      metrics/recall(B)'], 
                                                      row['       metrics/mAP50(B)'], 
                                                      row['    metrics/mAP50-95(B)']), axis=1)
        max_fitness_epoch = df['fitness'].idxmax()
        print(f" {max_precision_epoch} Max precision(B) with value {df['   metrics/precision(B)'][max_precision_epoch]}")
        print(f" {max_recall_epoch} Max recall(B) with value {df['      metrics/recall(B)'][max_recall_epoch]}")
        print(f" {max_mAP50_epoch} Max mAP50(B) with value {df['       metrics/mAP50(B)'][max_mAP50_epoch]}")
        print(f" {max_mAP50_95_epoch} Max mAP50-95(B) with value {df['    metrics/mAP50-95(B)'][max_mAP50_95_epoch]}")
        print(f" {max_fitness_epoch} Max fitness with value {df['fitness'][max_fitness_epoch]}")

yolov8_best:
 294 Max precision(B) with value 0.84462
 229 Max recall(B) with value 0.79974
 214 Max mAP50(B) with value 0.8619
 244 Max mAP50-95(B) with value 0.67489
 244 Max fitness with value 0.693402
yolov8_new_2132:
 239 Max precision(B) with value 0.85024
 289 Max recall(B) with value 0.79241
 234 Max mAP50(B) with value 0.85961
 244 Max mAP50-95(B) with value 0.67562
 239 Max fitness with value 0.69395
yolov8_new_2127:
 269 Max precision(B) with value 0.84148
 279 Max recall(B) with value 0.78381
 269 Max mAP50(B) with value 0.85695
 279 Max mAP50-95(B) with value 0.66988
 279 Max fitness with value 0.688535
yolov9:
 264 Max precision(B) with value 0.83879
 239 Max recall(B) with value 0.78492
 219 Max mAP50(B) with value 0.85061
 229 Max mAP50-95(B) with value 0.66348
 224 Max fitness with value 0.682153
yolov10:
 229 Max precision(B) with value 0.8231
 294 Max recall(B) with value 0.76354
 234 Max mAP50(B) with value 0.83321
 279 Max mAP50-95(B) with value 0.65034
 279 Max fi

Analyze activations

In [2]:
storage_path = Path("storage")

In [3]:
activation_files = {}
for f in storage_path.glob("*.pt"):
    # Filter files containing 'activations' in their name
    if "activations" in f.name:
        # Get the modification time of the file
        mod_time = f.stat().st_mtime
        # Convert modification time to a human-readable format, that is, to a date. Do not use torch
        # Convert modification time to a human-readable date
        readable_date = datetime.fromtimestamp(mod_time).strftime("%Y-%m-%d %H:%M:%S")
        # Almacena el nombre del archivo y su fecha legible
        activation_files[f.name] = readable_date
# Sort files by modification time in descending order
sorted_files = sorted(activation_files.items(), key=lambda x: x[1], reverse=True)
# Print the sorted file names
for file_name, mod_time in sorted_files:
    print(f"{file_name} - Last modified: {mod_time}")

logits_conf0.15_20250517_1728_owod_t1_yolo11l_from_scratch_activations_val_before_sigmoid.pt - Last modified: 2025-06-01 20:52:24
logits_conf0.15_20250517_1728_owod_t1_yolo11l_from_scratch_activations_before_sigmoid.pt - Last modified: 2025-06-01 20:50:23
logits_conf0.15_20250523_0011_owod_t1_yolo12l_from_scratch_activations_val_before_sigmoid.pt - Last modified: 2025-06-01 20:37:40
logits_conf0.15_20250523_0011_owod_t1_yolo12l_from_scratch_activations_before_sigmoid.pt - Last modified: 2025-06-01 20:36:24
ftmaps_and_strides_conf0.15_20250526_2123_owod_t1_yolov9m_from_scratch_activations_val_before_sigmoid.pt - Last modified: 2025-05-28 20:14:40
ftmaps_and_strides_conf0.15_20250526_2123_owod_t1_yolov9m_from_scratch_activations_before_sigmoid.pt - Last modified: 2025-05-28 20:10:20
logits_conf0.15_20250526_2123_owod_t1_yolov9m_from_scratch_activations_val_before_sigmoid.pt - Last modified: 2025-05-28 20:02:58
logits_conf0.15_20250526_2123_owod_t1_yolov9m_from_scratch_activations_before_

In [4]:
activation_file_yolov8 = "ftmaps_and_strides_conf0.15_20240313_1407_owod_t1_yolov8l_from_scratch_activations_before_sigmoid.pt"
activation_file_yolov9 = "ftmaps_and_strides_conf0.15_20250526_2123_owod_t1_yolov9m_from_scratch_activations_before_sigmoid.pt"
#activation_file_yolov10 = ""
activation_file_yolov11 = "ftmaps_and_strides_conf0.15_20250517_1728_owod_t1_yolo11l_from_scratch_activations_before_sigmoid.pt"
activation_file_yolov12 = "ftmaps_and_strides_conf0.15_20250523_0011_owod_t1_yolo12l_from_scratch_activations_before_sigmoid.pt"

# Load the activation files
activation_yolov8 = torch.load(storage_path / activation_file_yolov8)
activation_yolov9 = torch.load(storage_path / activation_file_yolov9)
# activation_yolov10 = torch.load(storage_path / activation_file_yolov10)
activation_yolov11 = torch.load(storage_path / activation_file_yolov11)
activation_yolov12 = torch.load(storage_path / activation_file_yolov12)

# Store them in a dictionary
activations_of_all_models = {
    "yolov8": activation_yolov8,
    "yolov9": activation_yolov9,
    # "yolov10": activation_yolov10,
    "yolov11": activation_yolov11,
    "yolov12": activation_yolov12,
}

In [5]:
print(len(activation_yolov8))
print(len(activation_yolov8[0]))
print(activation_yolov8[0][0].shape)
print(activation_yolov8[0][1].shape)
print(activation_yolov8[0][2].shape)

80
3
(243, 256, 1, 1)
(243, 512, 1, 1)
(670, 512, 1, 1)


In [6]:
def print_basic_statistics(activations):
    print(f"Num classes: {len(activations)}")
    print(f"Num strides: {len(activations[0])}")
    print(f"Shape of first activation: {activations[0][0].shape}")
    print(f"Shape of second activation: {activations[0][1].shape}")
    print(f"Shape of third activation: {activations[0][2].shape}")

In [7]:
for model_name, activation in activations_of_all_models.items():
    print(f"Statistics for {model_name}:")
    print_basic_statistics(activation)
    print()  # Print a newline for better readability

Statistics for yolov8:
Num classes: 80
Num strides: 3
Shape of first activation: (243, 256, 1, 1)
Shape of second activation: (243, 512, 1, 1)
Shape of third activation: (670, 512, 1, 1)

Statistics for yolov9:
Num classes: 80
Num strides: 3
Shape of first activation: (237, 240, 1, 1)
Shape of second activation: (232, 360, 1, 1)
Shape of third activation: (686, 480, 1, 1)

Statistics for yolov11:
Num classes: 80
Num strides: 3
Shape of first activation: (245, 256, 1, 1)
Shape of second activation: (244, 512, 1, 1)
Shape of third activation: (666, 512, 1, 1)

Statistics for yolov12:
Num classes: 80
Num strides: 3
Shape of first activation: (234, 256, 1, 1)
Shape of second activation: (204, 512, 1, 1)
Shape of third activation: (705, 512, 1, 1)



In [8]:
def print_advanced_statistics_for_certain_classes(activations_of_all_models, cls_idx_init, cls_idx_end):
    for model_name, activation in activations_of_all_models.items():
        print(f" **** {model_name} ****")
        for cls_idx, cls_activations in enumerate(activation):
            if cls_idx < cls_idx_init or cls_idx >= cls_idx_end:
                continue
            print(f"Class {cls_idx}:")
            for stride_idx, stride_activation in enumerate(cls_activations):
                print(f"  Stride {stride_idx}:")
                print(f"    Shape: {stride_activation.shape}")
                print(f"    Mean: {stride_activation.mean().item()}")
                print(f"    Std: {stride_activation.std().item()}")
                print(f"    Min: {stride_activation.min().item()}")
                print(f"    Max: {stride_activation.max().item()}")

In [9]:
print_advanced_statistics_for_certain_classes(activations_of_all_models, 14, 15)

 **** yolov8 ****
Class 14:
  Stride 0:
    Shape: (4436, 256, 1, 1)
    Mean: 0.8035409450531006
    Std: 1.63982093334198
    Min: -0.2781801223754883
    Max: 9.633785247802734
  Stride 1:
    Shape: (5484, 512, 1, 1)
    Mean: 0.3462652862071991
    Std: 1.0654213428497314
    Min: -0.2784646153450012
    Max: 6.925302505493164
  Stride 2:
    Shape: (2806, 512, 1, 1)
    Mean: 0.20058225095272064
    Std: 0.832394003868103
    Min: -0.27842962741851807
    Max: 5.63831090927124
 **** yolov9 ****
Class 14:
  Stride 0:
    Shape: (4177, 240, 1, 1)
    Mean: 1.265290379524231
    Std: 2.093419313430786
    Min: -0.2732011377811432
    Max: 11.4421968460083
  Stride 1:
    Shape: (5566, 360, 1, 1)
    Mean: 0.7441726326942444
    Std: 1.5467609167099
    Min: -0.278369665145874
    Max: 8.803767204284668
  Stride 2:
    Shape: (2888, 480, 1, 1)
    Mean: 0.3201407790184021
    Std: 0.9815523028373718
    Min: -0.2784422039985657
    Max: 5.597579002380371
 **** yolov11 ****
Class 14:


In [10]:
import plotly.graph_objects as go
import pandas as pd
from typing import List, Dict

def compare_activations_statistics_for_certain_classes(activations_of_all_models: Dict[str, List[List[np.ndarray]]], cls_idx, stride_idx):
    """
    This function compares the activations of different models for a specified class and for all dimensions individually of a certain stride.
    For each dimension, compute the mean and standard deviation of the activations.
    Makes a bar plot, where each dimension will have N bars, where N is the number of models. Each model will have its own color. 
    Plot it using plotly and make it an html.
    The shape of the activations is [n_objects, Dimensions, 1, 1] 
    """
    

    # Prepare data for the bar plot
    
    dimensions = activations_of_all_models[list(activations_of_all_models.keys())[0]][cls_idx][stride_idx].shape[1]
    print(f"Dimensions: {dimensions}")
    means = []
    stds = []
    model_names = []

    # Create a dataframe for means and other for stds, where the columns are the model names and the rows are the dimensions 
    for model_name, activations in activations_of_all_models.items():
        model_names.append(model_name)
        cls_activation = activations[cls_idx][stride_idx]
        
        # Calculate mean and std for each dimension
        mean = cls_activation.mean(axis=0).squeeze().tolist()
        std = cls_activation.std(axis=0).squeeze().tolist()
        means.append(mean)
        stds.append(std)

    # Create a DataFrame for means and stds
    df_means = pd.DataFrame(means, columns=[f'Dim {i+1}' for i in range(dimensions)], index=model_names)
    df_stds = pd.DataFrame(stds, columns=[f'Dim {i+1}' for i in range(dimensions)], index=model_names)

    # Convert means and stds to DataFrames for easier plotting
    means = pd.DataFrame(means)
    stds = pd.DataFrame(stds)

    # Create a bar plot where X axis is the dimension and Y axis is the mean of the activations. Each model is a color
    fig = go.Figure()
    for i, model_name in enumerate(model_names):
        fig.add_trace(go.Bar(
            x=[f'Dim {j+1}' for j in range(dimensions)],
            y=means.iloc[i],
            name=model_name,
            error_y=dict(type='data', array=stds.iloc[i], visible=True),
            marker_color=f'rgba({i * 50}, {100 + i * 30}, {150 + i * 20}, 0.6)'  # Different colors for each model
        ))
    fig.update_layout(
        title=f'Activation Comparison for Class {cls_idx} at Stride {stride_idx}',
        xaxis_title='Dimensions',
        yaxis_title='Mean Activation',
        barmode='group',
        template='plotly_white'
    )
        
    
    # Save the plot as an HTML file
    fig.write_html(f"activation_comparison_class_{cls_idx}_stride_{stride_idx}.html")

    return df_means, df_stds


In [11]:
activations_of_all_models["yolov8"][14][0].mean(axis=0).squeeze().shape

(256,)

In [12]:
df_m, df_s = compare_activations_statistics_for_certain_classes(activations_of_all_models, 14, 0)

Dimensions: 256


In [13]:
df_m

,Dim 1,Dim 2,Dim 3,Dim 4,Dim 5,Dim 6,Dim 7,Dim 8,Dim 9,Dim 10,...,Dim 247,Dim 248,Dim 249,Dim 250,Dim 251,Dim 252,Dim 253,Dim 254,Dim 255,Dim 256
yolov8,-0.069182,3.718833,0.273737,3.178309,0.175272,-0.240301,0.659785,-0.156254,1.670490,-0.008331,...,3.290478,-0.137236,0.293103,0.033975,-0.001741,0.031901,5.854834,-0.127098,-0.076530,-0.039140
yolov9,-0.099626,-0.250999,-0.138276,-0.084712,0.662590,-0.182916,6.993863,0.067692,3.522442,0.230664,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
yolov11,-0.028110,-0.029244,0.579612,-0.048450,-0.159864,0.127102,-0.004555,-0.048504,-0.166237,0.333507,...,6.258946,4.953309,0.070075,5.223000,0.033858,-0.143313,1.839896,4.729328,0.503620,2.905619
yolov12,0.164076,0.168594,-0.010455,-0.140272,-0.003273,-0.149620,2.267666,-0.003323,1.087793,-0.179693,...,0.070006,4.827345,3.838210,7.971994,0.564905,0.261700,4.554006,0.266123,-0.040844,0.270000
